In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [18]:
data = pd.read_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/Clean4Model')
data = data.drop(columns=['Unnamed: 0'])

In [19]:
# Features and target
X = data.drop(columns=['Country'])
y = data['Country']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [21]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
model = BalancedRandomForestClassifier(n_estimators=100, random_state=42, bootstrap=False, replacement = True)
model.fit(X_train_scaled, y_train)

/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(


BalancedRandomForestClassifier(bootstrap=False, random_state=42,
                               replacement=True)

In [23]:
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7516602331618812

Classification Report:
               precision    recall  f1-score   support

           1       0.93      0.92      0.92    106603
           2       0.72      0.49      0.58     66336
           3       0.59      0.69      0.63     38184
           4       0.91      0.84      0.87     34085
           5       0.67      0.71      0.69     17412
           6       0.52      0.65      0.57     15252
           7       0.51      0.88      0.65      9434
           8       0.67      0.74      0.70      7687
           9       0.81      0.88      0.85      7008
          10       0.36      0.69      0.48      6540

    accuracy                           0.75    308541
   macro avg       0.67      0.75      0.69    308541
weighted avg       0.77      0.75      0.75    308541


Confusion Matrix:
 [[97933   585    13     6  1545    15  3480    49     7  2970]
 [ 4461 32272 14421  1672  1601  3659  4183  1086   531  2450]
 [ 1247  1542 26234   794  2584  3743   1

In [25]:

from scipy.stats import randint

param_distributions = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 50],
    'max_features': ['sqrt', 'log2', 0.3, 0.5, None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'bootstrap': [True, False]
}

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=30,
    scoring='f1_macro',  # or 'roc_auc', 'balanced_accuracy'
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

search.fit(X_train, y_train)
print("Best Params:", search.best_params_)
print("Best Score:", search.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.


Best Params: {'bootstrap': False, 'max_depth': 20, 'max_features': 0.5, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 419}
Best Score: 0.7348381509357373
